In [1]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import pandas as pd
import io
import os
import datetime
import config
import encoding_decoding
from sqlalchemy import create_engine
import blob_helper

In [2]:
#Engine to upload dataframe to empty table in mysql
engine = create_engine(config.engine)

In [3]:
#creating container in azure storage
blob_helper.create_container(config.container_name)

'container created'

In [4]:
#uploading file to azure storage container
file_to_upload = "aman.txt"
blob_helper.upload_file(config.container_name, file_to_upload, "aman.txt")

'file uploaded successfully'

In [5]:
# List all containers in the storage account
blob_helper.list_container_name()

sales-project


In [6]:
#list all blobs inside a container
blob_helper.list_blob_in_container(config.container_name)

aman.txt
customers_datamart.csv
sales_data
sales_data.csv
sales_team_datamart.csv


In [7]:
#check if file is present or not in local directory
directory_path = config.sales_dir_path
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

In [8]:
if csv_files:
    print("CSV files found in the directory:")
    for csv_file in csv_files:
        print(csv_file)
else:
    print("No CSV files found in the directory.")

CSV files found in the directory:
sales_data.csv


In [11]:
# Upload a file to the container
f_to_upload = directory_path+"\\"+csv_file
blob_helper.upload_file(config.container_name,f_to_upload,"sales_data.csv")

'file uploaded successfully'

In [13]:
#Download same file to local storage
csv_file_name = config.master_file_name
df = blob_helper.read_csv_from_blob(config.container_name, csv_file_name)

In [14]:
df.head()

,customer_id,store_id,product_name,sales_date,sales_person_id,price,quantity,total_cost
0,19,123,clinic plus,2023-04-03,9,1.5,7,10.5
1,2,122,maida,2023-05-11,6,20.0,4,80.0
2,15,122,dantkanti,2023-07-04,4,100.0,2,200.0
3,8,122,maida,2023-08-07,6,20.0,1,20.0
4,17,121,sugar,2023-05-03,1,50.0,10,500.0


In [15]:
# MySQL database connection parameters
host = config.host
user = config.user
password = config.password
database = config.database

In [16]:
# Create a MySQL database connection
connection = blob_helper.get_mysql_connection()

In [17]:
#mysql to dataframe
def load(table_name):
    query = f"SELECT * FROM "+table_name
    return query

In [18]:
# Query to retrieve data from the MySQL table
#query = f"SELECT * FROM product"
# Read data into a Pandas DataFrame
df_store = pd.read_sql(load("store"), connection)
df_cust = pd.read_sql(load("customer"), connection)
df_sales = pd.read_sql(load("sales_team"), connection)
df_prod = pd.read_sql(load("product"), connection)

C:\Users\aman.ab.srivastava\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\aman.ab.srivastava\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\aman.ab.srivastava\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\aman.ab.srivastava\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connecta

In [19]:
df_cust.head()

,customer_id,first_name,last_name,address,pincode,phone_number,customer_joining_date
0,1,Saanvi,Krishna,Delhi,122009,9173121081,2021-01-20
1,2,Dhanush,Sahni,Delhi,122009,9155328165,2022-03-27
2,3,Yasmin,Shan,Delhi,122009,9191478300,2023-04-08
3,4,Vidur,Mammen,Delhi,122009,9119017511,2020-10-12
4,5,Shamik,Doctor,Delhi,122009,9105180499,2022-10-30


In [20]:
df_sales.head()

,id,first_name,last_name,manager_id,is_manager,address,pincode,joining_date
0,1,Rahul,Verma,10,N,Delhi,122007,2020-05-01
1,2,Priya,Singh,10,N,Delhi,122007,2020-05-01
2,3,Amit,Sharma,10,N,Delhi,122007,2020-05-01
3,4,Sneha,Gupta,10,N,Delhi,122007,2020-05-01
4,5,Neha,Kumar,10,N,Delhi,122007,2020-05-01


In [21]:
df_prod.head()

,id,name,current_price,old_price,created_date,updated_date,expiry_date
0,1,quaker oats,212.0,212.0,2022-05-15,None,2025-01-01
1,2,sugar,50.0,50.0,2021-08-10,None,2025-01-01
2,3,maida,20.0,20.0,2023-03-20,None,2025-01-01
3,4,besan,52.0,52.0,2020-05-05,None,2025-01-01
4,5,refined oil,110.0,110.0,2022-01-15,None,2025-01-01


In [22]:
df.head()

,customer_id,store_id,product_name,sales_date,sales_person_id,price,quantity,total_cost
0,19,123,clinic plus,2023-04-03,9,1.5,7,10.5
1,2,122,maida,2023-05-11,6,20.0,4,80.0
2,15,122,dantkanti,2023-07-04,4,100.0,2,200.0
3,8,122,maida,2023-08-07,6,20.0,1,20.0
4,17,121,sugar,2023-05-03,1,50.0,10,500.0


In [23]:
res_df= pd.merge(df, df_cust, on= 'customer_id', how='inner')
res_df= pd.merge(res_df, df_prod , left_on= 'product_name',right_on = 'name', how='inner')

In [24]:
res_df= pd.merge(res_df, df_sales , left_on= 'sales_person_id',right_on = 'id', how='inner')

In [25]:
res_df.head()

,customer_id,store_id,product_name,sales_date,sales_person_id,price,quantity,total_cost,first_name_x,last_name_x,...,updated_date,expiry_date,id_y,first_name_y,last_name_y,manager_id,is_manager,address_y,pincode_y,joining_date
0,19,123,clinic plus,2023-04-03,9,1.5,7,10.5,Indranil,Dutta,...,None,2025-01-01,9,Monica,Jain,10,N,Delhi,122007,2020-05-01
1,7,123,clinic plus,2023-04-05,9,1.5,6,9.0,Romil,Shanker,...,None,2025-01-01,9,Monica,Jain,10,N,Delhi,122007,2020-05-01
2,1,123,clinic plus,2023-06-11,9,1.5,9,13.5,Saanvi,Krishna,...,None,2025-01-01,9,Monica,Jain,10,N,Delhi,122007,2020-05-01
3,18,123,clinic plus,2023-08-16,9,1.5,3,4.5,Vardaniya,Jani,...,None,2025-01-01,9,Monica,Jain,10,N,Delhi,122007,2020-05-01
4,5,123,clinic plus,2023-06-15,9,1.5,1,1.5,Shamik,Doctor,...,None,2025-01-01,9,Monica,Jain,10,N,Delhi,122007,2020-05-01


In [26]:
df_store

,id,address,store_pincode,store_manager_name,store_opening_date,reviews
0,121,Delhi,122009,Manish,2022-01-15,Great store with a friendly staff.
1,122,Delhi,110011,Nikita,2021-08-10,Excellent selection of products.
2,123,Delhi,201301,vikash,2023-01-20,Clean and organized store.
3,124,Delhi,400001,Rakesh,2020-05-05,Good prices and helpful staff.


In [27]:
res_df= pd.merge(res_df, df_store , left_on= 'store_id',right_on = 'id', how='inner')

In [28]:
res_df.head()

,customer_id,store_id,product_name,sales_date,sales_person_id,price,quantity,total_cost,first_name_x,last_name_x,...,is_manager,address_y,pincode_y,joining_date,id,address,store_pincode,store_manager_name,store_opening_date,reviews
0,19,123,clinic plus,2023-04-03,9,1.5,7,10.5,Indranil,Dutta,...,N,Delhi,122007,2020-05-01,123,Delhi,201301,vikash,2023-01-20,Clean and organized store.
1,7,123,clinic plus,2023-04-05,9,1.5,6,9.0,Romil,Shanker,...,N,Delhi,122007,2020-05-01,123,Delhi,201301,vikash,2023-01-20,Clean and organized store.
2,1,123,clinic plus,2023-06-11,9,1.5,9,13.5,Saanvi,Krishna,...,N,Delhi,122007,2020-05-01,123,Delhi,201301,vikash,2023-01-20,Clean and organized store.
3,18,123,clinic plus,2023-08-16,9,1.5,3,4.5,Vardaniya,Jani,...,N,Delhi,122007,2020-05-01,123,Delhi,201301,vikash,2023-01-20,Clean and organized store.
4,5,123,clinic plus,2023-06-15,9,1.5,1,1.5,Shamik,Doctor,...,N,Delhi,122007,2020-05-01,123,Delhi,201301,vikash,2023-01-20,Clean and organized store.


In [29]:
res_df.columns

Index(['customer_id', 'store_id', 'product_name', 'sales_date',
       'sales_person_id', 'price', 'quantity', 'total_cost', 'first_name_x',
       'last_name_x', 'address_x', 'pincode_x', 'phone_number',
       'customer_joining_date', 'id_x', 'name', 'current_price', 'old_price',
       'created_date', 'updated_date', 'expiry_date', 'id_y', 'first_name_y',
       'last_name_y', 'manager_id', 'is_manager', 'address_y', 'pincode_y',
       'joining_date', 'id', 'address', 'store_pincode', 'store_manager_name',
       'store_opening_date', 'reviews'],
      dtype='object')

In [30]:
col= ['customer_id','first_name_x','last_name_x','store_id','address','store_pincode','store_manager_name','product_name','current_price','expiry_date','sales_date','sales_person_id','first_name_y','last_name_y','joining_date','price','quantity','total_cost']

In [31]:
res_df= res_df[col]

In [32]:
res_df.head()

,customer_id,first_name_x,last_name_x,store_id,address,store_pincode,store_manager_name,product_name,current_price,expiry_date,sales_date,sales_person_id,first_name_y,last_name_y,joining_date,price,quantity,total_cost
0,19,Indranil,Dutta,123,Delhi,201301,vikash,clinic plus,1.5,2025-01-01,2023-04-03,9,Monica,Jain,2020-05-01,1.5,7,10.5
1,7,Romil,Shanker,123,Delhi,201301,vikash,clinic plus,1.5,2025-01-01,2023-04-05,9,Monica,Jain,2020-05-01,1.5,6,9.0
2,1,Saanvi,Krishna,123,Delhi,201301,vikash,clinic plus,1.5,2025-01-01,2023-06-11,9,Monica,Jain,2020-05-01,1.5,9,13.5
3,18,Vardaniya,Jani,123,Delhi,201301,vikash,clinic plus,1.5,2025-01-01,2023-08-16,9,Monica,Jain,2020-05-01,1.5,3,4.5
4,5,Shamik,Doctor,123,Delhi,201301,vikash,clinic plus,1.5,2025-01-01,2023-06-15,9,Monica,Jain,2020-05-01,1.5,1,1.5


In [33]:
df.to_csv(config.master_dir+"\\master_df.csv", index=False)

In [34]:
#Basic Transformation 1
df.head()

,customer_id,store_id,product_name,sales_date,sales_person_id,price,quantity,total_cost
0,19,123,clinic plus,2023-04-03,9,1.5,7,10.5
1,2,122,maida,2023-05-11,6,20.0,4,80.0
2,15,122,dantkanti,2023-07-04,4,100.0,2,200.0
3,8,122,maida,2023-08-07,6,20.0,1,20.0
4,17,121,sugar,2023-05-03,1,50.0,10,500.0


In [35]:
df['month'] = -1
for i in range(len(df)):
    df['month'][i] = datetime.datetime.strptime(df['sales_date'][i], "%Y-%m-%d").month

C:\Users\aman.ab.srivastava\AppData\Local\Temp\ipykernel_14848\1833660722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'][i] = datetime.datetime.strptime(df['sales_date'][i], "%Y-%m-%d").month


In [36]:
df.head()

,customer_id,store_id,product_name,sales_date,sales_person_id,price,quantity,total_cost,month
0,19,123,clinic plus,2023-04-03,9,1.5,7,10.5,4
1,2,122,maida,2023-05-11,6,20.0,4,80.0,5
2,15,122,dantkanti,2023-07-04,4,100.0,2,200.0,7
3,8,122,maida,2023-08-07,6,20.0,1,20.0,8
4,17,121,sugar,2023-05-03,1,50.0,10,500.0,5


In [37]:
# Group by customer_id and month, and calculate the total sales
result = df.groupby(['customer_id', 'month']).agg(
    total_sales=pd.NamedAgg(column='price', aggfunc='sum'),
    store_id=pd.NamedAgg(column='store_id', aggfunc='first'),
    product_name=pd.NamedAgg(column='product_name', aggfunc='first'),
    sales_person_id=pd.NamedAgg(column='sales_person_id', aggfunc='first'),
    quantity=pd.NamedAgg(column='quantity', aggfunc='first')
).reset_index()

In [38]:
result.head()

,customer_id,month,total_sales,store_id,product_name,sales_person_id,quantity
0,1,3,222.0,122,sugar,5,3
1,1,4,395.0,123,dantkanti,7,3
2,1,5,312.0,123,dantkanti,7,4
3,1,6,415.0,123,clinic plus,7,3
4,1,7,123.5,123,besan,8,2


In [39]:
#push this to mysql as customer_data_mart

In [40]:
cust_dm = pd.merge(result,df_cust, on='customer_id', how = 'inner')

In [41]:
cust_dm['full_name'] = cust_dm['first_name']+' '+cust_dm['last_name'] 

In [42]:
cust_dm.head()

,customer_id,month,total_sales,store_id,product_name,sales_person_id,quantity,first_name,last_name,address,pincode,phone_number,customer_joining_date,full_name
0,1,3,222.0,122,sugar,5,3,Saanvi,Krishna,Delhi,122009,9173121081,2021-01-20,Saanvi Krishna
1,1,4,395.0,123,dantkanti,7,3,Saanvi,Krishna,Delhi,122009,9173121081,2021-01-20,Saanvi Krishna
2,1,5,312.0,123,dantkanti,7,4,Saanvi,Krishna,Delhi,122009,9173121081,2021-01-20,Saanvi Krishna
3,1,6,415.0,123,clinic plus,7,3,Saanvi,Krishna,Delhi,122009,9173121081,2021-01-20,Saanvi Krishna
4,1,7,123.5,123,besan,8,2,Saanvi,Krishna,Delhi,122009,9173121081,2021-01-20,Saanvi Krishna


In [43]:
cust_dm.columns

Index(['customer_id', 'month', 'total_sales', 'store_id', 'product_name',
       'sales_person_id', 'quantity', 'first_name', 'last_name', 'address',
       'pincode', 'phone_number', 'customer_joining_date', 'full_name'],
      dtype='object')

In [44]:
cust_dm = cust_dm[['customer_id','full_name', 'address','phone_number','month', 'total_sales']]

In [45]:
cust_dm = cust_dm.rename(columns = {'month_x': 'sales_date_month'})

In [46]:
cust_dm.head()

,customer_id,full_name,address,phone_number,month,total_sales
0,1,Saanvi Krishna,Delhi,9173121081,3,222.0
1,1,Saanvi Krishna,Delhi,9173121081,4,395.0
2,1,Saanvi Krishna,Delhi,9173121081,5,312.0
3,1,Saanvi Krishna,Delhi,9173121081,6,415.0
4,1,Saanvi Krishna,Delhi,9173121081,7,123.5


In [47]:
cust_dm.to_sql('customers_data_mart', con=engine, if_exists='replace', index=False)

118

In [48]:
#Basic Transformation 2
df_sales.head()

,id,first_name,last_name,manager_id,is_manager,address,pincode,joining_date
0,1,Rahul,Verma,10,N,Delhi,122007,2020-05-01
1,2,Priya,Singh,10,N,Delhi,122007,2020-05-01
2,3,Amit,Sharma,10,N,Delhi,122007,2020-05-01
3,4,Sneha,Gupta,10,N,Delhi,122007,2020-05-01
4,5,Neha,Kumar,10,N,Delhi,122007,2020-05-01


In [49]:
result2 = df.groupby(['store_id','sales_person_id','month']).agg(
    total_sales=pd.NamedAgg(column='price', aggfunc='sum'),
    customer_id=pd.NamedAgg(column='customer_id', aggfunc='sum'),
    #store_id=pd.NamedAgg(column='store_id', aggfunc='first'),
    product_name=pd.NamedAgg(column='product_name', aggfunc='first'),
    quantity=pd.NamedAgg(column='quantity', aggfunc='first')
).reset_index()

In [50]:
result2.head()

,store_id,sales_person_id,month,total_sales,customer_id,product_name,quantity
0,121,1,3,206.5,53,clinic plus,4
1,121,1,4,535.0,100,refined oil,5
2,121,1,5,768.0,118,sugar,10
3,121,1,6,1261.5,101,clinic plus,4
4,121,1,7,650.0,94,dantkanti,8


In [51]:
result2[result2.store_id == 121]

,store_id,sales_person_id,month,total_sales,customer_id,product_name,quantity
0,121,1,3,206.5,53,clinic plus,4
1,121,1,4,535.0,100,refined oil,5
2,121,1,5,768.0,118,sugar,10
3,121,1,6,1261.5,101,clinic plus,4
4,121,1,7,650.0,94,dantkanti,8
5,121,1,8,1160.0,93,quaker oats,3
6,121,2,3,786.0,74,maida,1
7,121,2,4,805.5,97,quaker oats,10
8,121,2,5,604.0,86,refined oil,9
9,121,2,6,726.0,83,nutrella,7


In [52]:
# Calculate the salesperson with the most sales per store_id and month
max_sales_person = result2.groupby(['store_id', 'month'])['total_sales'].idxmax()
result2['bonus'] = 0.01 * result2['total_sales'] * result2.index.isin(max_sales_person).astype(int)


In [53]:
result2

,store_id,sales_person_id,month,total_sales,customer_id,product_name,quantity,bonus
0,121,1,3,206.5,53,clinic plus,4,0.000
1,121,1,4,535.0,100,refined oil,5,0.000
2,121,1,5,768.0,118,sugar,10,0.000
3,121,1,6,1261.5,101,clinic plus,4,12.615
4,121,1,7,650.0,94,dantkanti,8,0.000
5,121,1,8,1160.0,93,quaker oats,3,11.600
6,121,2,3,786.0,74,maida,1,7.860
7,121,2,4,805.5,97,quaker oats,10,0.000
8,121,2,5,604.0,86,refined oil,9,0.000
9,121,2,6,726.0,83,nutrella,7,0.000


In [54]:
sales_dm = pd.merge(result2,df_sales, left_on='sales_person_id', right_on='id', how = 'inner')

In [55]:
sales_dm['full_name'] = sales_dm['first_name']+' '+sales_dm['last_name']

In [56]:
sales_dm.columns

Index(['store_id', 'sales_person_id', 'month', 'total_sales', 'customer_id',
       'product_name', 'quantity', 'bonus', 'id', 'first_name', 'last_name',
       'manager_id', 'is_manager', 'address', 'pincode', 'joining_date',
       'full_name'],
      dtype='object')

In [57]:
sales_dm = sales_dm[['store_id', 'sales_person_id','full_name', 'month', 'total_sales','bonus'
       ]]

In [58]:
sales_dm = sales_dm.sort_values(by= ['store_id', 'month','bonus'], ascending=[True,True,False])

In [59]:
sales_dm = sales_dm.rename(columns={'month':'sales_month', 'bonus':'incentive'})

In [60]:
sales_dm.head()

,store_id,sales_person_id,full_name,sales_month,total_sales,incentive
6,121,2,Priya Singh,3,786.0,7.860
0,121,1,Rahul Verma,3,206.5,0.000
12,121,3,Amit Sharma,3,396.0,0.000
13,121,3,Amit Sharma,4,819.5,8.195
1,121,1,Rahul Verma,4,535.0,0.000


In [61]:
#!pip install mysqlclient

In [62]:
sales_dm.to_sql('sales_team_data_mart', con=engine, if_exists='replace', index=False)

54

In [63]:
#sales_dm.to_csv("sales_team_datamart.csv", index=False)

In [66]:
f_to_upload2 = "C:\\Users\\aman.ab.srivastava\\Documents\\pro\\data\\sales_datamart\\sales_team_datamart.csv"
# blob_client2 = blob_service_client.get_blob_client(container_name, "sales_team_datamart.csv")
# with open(f_to_upload2, "rb") as data:
#     blob_client2.upload_blob(data, overwrite=True)
    
blob_helper.upload_file(config.container_name, f_to_upload2, "sales_team_datamart.csv")    

'file uploaded successfully'

In [67]:
#cust_dm.to_csv("customers_datamart.csv", index=False)

In [68]:
f_to_upload3 = "C:\\Users\\aman.ab.srivastava\\Documents\\pro\\data\\customer_datamart\\customers_datamart.csv"

In [71]:
blob_helper.upload_file(config.container_name, f_to_upload3, "customers_datamart.csv")    

'file uploaded successfully'